In [2]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25857 sha256=7573fe06c8092ee593984ea0524ffa19d565a1bd15f49715b2ed8719cfc9bd7b
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from imutils.object_detection import non_max_suppression
import os

WINDOW SLIDE

    滑动窗口方法：遍历图像并裁剪小窗口

    :param image: 输入图像

    :param step_size: 滑动步长

    :param window_size: 窗口大小 (height, width)

    :return: 生成器，返回窗口坐标 (x, y) 和窗口区域 (ROI)

In [ ]:
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            yield (x, y, image[y:y + window_size[1], x + window_size[0]])

Load precreated crater_cnn_model

In [ ]:
model = load_model("improved_crater_cnn.keras") 

 Read the image and perform window detection

In [ ]:
image_path = r"D:\NOTEBOOK\FYP\NASADataset\1.jpg"
image = cv2.imread(image_path)  # Read the image
if image is None:
    raise ValueError("Failed to load the image, please check the path!")

# Preprocessing
image_resized = cv2.resize(image, (512, 512))  # Resize to a uniform size
gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)  # CNN requires 3-channel input

window_size = (64, 64)  # Input size required by the CNN
step_size = 32  # Sliding window step size

rects = []
scores = []
# Iterate over all windows and perform CNN prediction
for (x, y, window) in sliding_window(gray, step_size, window_size):
    if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
        continue  # Ignore incomplete windows

    # Preprocess the window and perform CNN prediction
    window = window.astype("float32") / 255.0  # Normalize
    window = np.expand_dims(window, axis=0)  # Add batch dimension
    
    # CNN prediction, returns confidence score
    prob = model.predict(window)[0][0]
    
    if prob > 0.5:  # Set a threshold to filter regions likely containing craters
        rects.append((x, y, x + window_size[0], y + window_size[1]))
        scores.append(prob)

Perform Non-Maximum Suppression (NMS) to merge overlapping windows(OpenCV's)

In [ ]:
rects = np.array(rects)
scores = np.array(scores)
final_boxes = non_max_suppression(rects, probs=scores, overlapThresh=0.3)

Generate YOLO format data: <class_id> <x_center> <y_center> <width> <height>

In [ ]:
yolo_annotations = []
for (x1, y1, x2, y2) in final_boxes:
    img_h, img_w = image_resized.shape[:2]

    # normalized
    x_center = (x1 + x2) / 2.0 / img_w
    y_center = (y1 + y2) / 2.0 / img_h
    box_width = (x2 - x1) / img_w
    box_height = (y2 - y1) / img_h

    yolo_annotations.append(f"0 {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")

# Save
output_txt_path = image_path.replace(".jpg", ".txt")
with open(output_txt_path, "w") as f:
    f.write("\n".join(yolo_annotations))

print(f"YOLO annotation file saved: {output_txt_path}")